<a href="https://colab.research.google.com/github/Viny2030/HUMAI/blob/main/agrupacion_agregacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/AnalisisDeDatos/3_Agrupacion_y_Agregacion/agrupacion_agregacion.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>
<div align="center"> Recordá abrir en una nueva pestaña </div>

# Agrupación


## Tabla de Contenidos

    I. Agrupación y agregación
        I. Introducción
            I. Crear columnas
    III. Funciones de Agregación simple
    IV. GroupBy: Trabajando sobre grupos
        I. Clases que se encargan de la Agregación en Pandas
        II. DataSetGroupBy
            I. Lazy Evaluation
            II. Iterar sobre los grupos
        III. SeriesGroupBy
        IV. Agregaciones múltiples
        V. MultiIndex
        VI. Ejercicios

## Introducción

En esta clase vamos a ver cómo agrupar datos a partir de una clave para trabajar sobre distintos grupos.
El primer dataset que vamos a utilizar es del portal de datos abiertos de España.  Los datos se pueden encontrar <a href='http://opendata.esri.es/datasets/paro-por-municipio-espa%C3%B1a/geoservice'> aquí</a> y la única modificación que se les hizo fue reemplazar el código de provincia por la descripción de la misma

In [1]:
! pip install plotly

In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df = pd.read_csv('https://storage.googleapis.com/humai-datasets/aws_s3/analisis_de_datos/3_Agrupacion_y_Agregacion/parodesprov.csv')

In [4]:
df.sort_values('Codigo').head(6)

,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop
0,1,1001,Alegría-Dulantzi,1,16,184.0,2856.0,Araba/Álava,16.0,3.709269e+07,48191.568537,1.0
8170,8171,1001,Alegría-Dulantzi,1,16,185.0,2913.0,NaN,NaN,3.709269e+07,48191.568751,NaN
1,2,1002,Amurrio,1,16,766.0,10260.0,Araba/Álava,16.0,1.799114e+08,90623.621986,1.0
8171,8172,1002,Amurrio,1,16,766.0,10291.0,NaN,NaN,1.799114e+08,90623.621773,NaN
2,3,1003,Aramaio,1,16,18.0,1502.0,Araba/Álava,16.0,1.377066e+08,58592.914048,1.0
8172,8173,1003,Aramaio,1,16,18.0,1496.0,NaN,NaN,1.377066e+08,58592.915252,NaN


Noten que en este conjunto de datos, al parecer la base está duplicada. Los municipios figuran dos veces, una vez con el código de provincia en NaN y otra vez con el código de provincia informado. Los valores de paro y población son muy cercanos en todos los casos, así que podemos considerarlas cercanas en el tiempo.

In [5]:
df = df[df['PAD_1_COD_PROV'].notnull()].copy()

## Niveles de Agregación

Además de los municipios (identificados por la variable Código), tenemos dos niveles de agregación geográfica, la provincia (Cod_Prov) y la comunidad autónoma (Cod_CAA).

Veamos qué valores toman y cómo se combinan.


In [6]:
# ¿Cuántos municipios tiene cada comunidad? ¿Hay alguno que no tenga CCAA asociada?
df['Cod_CCAA'].value_counts(dropna=False)

,count
Cod_CCAA,
7,2248
9,947
8,919
1,778
2,731
10,542
11,388
12,314
15,272


In [7]:
# ¿Cuántas CCAA hay?
len(df['Cod_CCAA'].unique())

19

In [8]:
# ¿Cuántos municipios tiene cada provincia? ¿Hay alguno que no tenga provincia asociada?
df['Cod_Prov'].value_counts(dropna=False)

,count
Cod_Prov,
9,371
37,362
8,311
50,293
19,288
31,272
46,266
5,248
49,248


In [9]:
# ¿Cuántas provincias hay?
len(df['Cod_Prov'].unique())

52

In [10]:
# ¿Hay alguna provincia que tenga más de una CCAA asociada?
df[['Cod_CCAA','Cod_Prov']].drop_duplicates().sort_values('Cod_Prov')

,Cod_CCAA,Cod_Prov
0,16,1
51,8,2
138,10,3
279,1,4
382,7,5
630,11,6
795,4,7
862,9,8
1173,7,9
1544,11,10


## Buscando valores nulos

Ahora queremos filtrar todas las filas del DataFrame que contengan algún valor nulo, para analizar en detalle por qué existe ese valor faltante y si vamos a querer completarlo o descartarlo.

El método df.isnull() devuelve un DataFrame del mismo tamaño que el original, pero en lugar de los valores devuelve True si había un NaN y False si no lo había.

In [11]:
df.isnull().head(3)

,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False


#### Entendiendo el parámetro axis
A continuación vamos a reducir el DataFrame de más arriba a una serie, compuesta por Booleans que indican True si hay algún valor nulo en la fila y False si no hay ninguno. La forma de reducir una serie de Booleanos es any() y el parámetro axis = 1 indica que queremos reducir el DataFrame aplicando la función de manera horizontal, probando todos los valores del eje y.



In [12]:
df[df.isnull().any(axis=1)]


,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop
1766,1767,10905,Pueblonuevo de Miramontes,10,11,NaN,830.0,Cáceres,11.0,3.979055e+07,34532.621933,10.0


Otro posible método para reucir es all así que una forma equivalente de lograr lo mismo es:

In [13]:
df[~(df.notnull().all(axis=1))]  ## operador negacion no sea cierto ningun valor nulo


,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop
1766,1767,10905,Pueblonuevo de Miramontes,10,11,NaN,830.0,Cáceres,11.0,3.979055e+07,34532.621933,10.0


¿Qué significará un TotalParoRegistrado nulo? :thinking: ¿Será equivalente a un paro de 0, algo que es posible que se de?

In [14]:
df[df['TotalParoRegistrado'] == 0].sample(3)

,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop
3048,3049,19238,Riofrío del Llano,19,8,0.0,46.0,Guadalajara,8.0,7.573516e+07,41310.720915,19.0
3113,3114,19309,Valhermoso,19,8,0.0,30.0,Guadalajara,8.0,5.078280e+07,34517.082063,19.0
4806,4807,31196,Orbara,31,15,0.0,40.0,Navarra,15.0,1.709542e+07,19580.146675,31.0


Podríamos concluir que no, porque los municipios con paro igual a 0, informan 0. Entonces deberíamos descartar el dato.

#### Entendiendo el parámetro inplace

Noten que la mayor parte de los métodos que trabajan sobre DataFrames devuelven objetos nuevos que si no los almacenamos en una variable se pierden. Cuando queremos que el DataFrame cambie a partir de una determinada acción usamos el parámetro inplace=True.

In [15]:
df.dropna(inplace=True)  ## descarta todas las filas que tengan valor nulo

In [16]:
df[~(df.notnull().all(axis=1))]

,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop


### Crear columnas

Contamos con los cambios de población de cada municipio ('PAD_1C02') y también con el área ('Shape__Area'). Con estas columnas podemos formar la densidad.

In [17]:
# ¡Operación vectorizada!
df['Densidad'] = df['PAD_1C02'] / df['Shape__Area']

In [18]:
df.Densidad

,Densidad
0,7.699629e-05
1,5.702806e-05
2,1.090725e-05
3,3.586690e-05
4,9.830741e-06
...,...
8165,5.967305e-07
8166,2.933494e-06
8167,1.691438e-05
8168,2.870696e-03


También sabemos la cantidad de personas desempleadas y con eso podemos formar la proporción de paro.

In [19]:
df['Proporcion_Paro'] =  df['TotalParoRegistrado'] / df['PAD_1C02']

In [20]:
df.Proporcion_Paro

,Proporcion_Paro
0,0.064426
1,0.074659
2,0.011984
3,0.059783
4,0.051282
...,...
8165,0.020979
8166,0.021978
8167,0.034145
8168,0.147624


## Funciones de Agregación simple


| Nombre             | Versión que descarta NaN | Descripción                            |
|--------------------|--------------------------|----------------------------------------|
| serie.sum()        | serie.sum(skipna=True)   | Suma todos los elementos               |
| serie.prod()       | serie.prod(skipna=True)  | Multiplica                             |
| serie.mean()       | serie.mean(skipna=True)  | Promedia                               |
| serie.std()        | serie.std(skipna=True)   | Calcula el desvío estándar             |
| serie.var()        | serie.var(skipna=True)   | Calcula la varianza                    |
| serie.min()        | serie.min(skipna=True)   | Calcula el valor mínimo                |
| serie.max()        | serie.max(skipna=True)   | Calcula el valor máximo                |
| serie.argmin()     | serie.argmin(skipna=True)| Calcula el índice del valor mínimo     |
| serie.argmax()     | serie.argmax(skipna=True)| Calcula el índice del valor máximo     |
| serie.median()     | serie.median(skipna=True)| Calcula la mediana                     |
| X                  | serie.quantile()         | Calcula los percentiles                |
| serie.any()        | X                        | Evalúa si algún elemento es TRUE       |
| serie.all()        | X                        | Evalúa si todos los elementos son TRUE |

Vamos a comprobar que la población total coincida (aproxiamadamente) con la <a href='https://www.google.com/search?client=firefox-b-e&q=poblacion+espana'>población de España.  <a/>

In [21]:
df['PAD_1C02'].sum()

46556178.0

¿Cuál es el promedio de proporción de paro en las autonomías de España?

In [22]:
df['Proporcion_Paro'].mean()

0.06017574475360859

Ejercicio: ¿Cuántas autonomías están por encima de la media y cuántas están por debajo?

In [23]:
### Ejercicio: ¿Cuántas autonomías están por encima de la media y cuántas están por debajo?
df[df['Proporcion_Paro'] > df['Proporcion_Paro'].mean()].shape[0]

3688

In [24]:
df[df['Proporcion_Paro'] < df['Proporcion_Paro'].mean()].shape[0]

4436

¿Cuál es la mediana?

In [25]:
df['Proporcion_Paro'].mean()

0.06017574475360859

In [26]:
## ¿Cuál es la mediana?
df['Proporcion_Paro'].median()

0.05660377358490566

Que la mediana sea menor a la media, significa que hay algunos valores llamativamente altos. Veamos un gráfico para ver la distribución de esta variable.

In [27]:
fig = px.imshow (df[["Proporcion_Paro"]])
fig.show()

In [28]:
fig = px.box (df, x="Proporcion_Paro")
fig.show()

In [29]:
fig = px.histogram(df, x="Proporcion_Paro")
fig.show()

Y en cuanto al tamaño, ¿cuál es la mediana de tamaño en las autonomías?

In [30]:
mediana_area = df['Shape__Area'].median()

In [31]:
mediana_area

61403802.64746095

In [32]:
mediana_densidad = df['Densidad'].median()
mediana_densidad

7.903238601327104e-06

Si dividimos a las autonomías entre "grandes" (con un área mayor que la mediana) y "chicas" (con un área menor), ¿cuál grupo tiene mayor proporción de paro?

In [33]:
df.query('Shape__Area > @mediana_area')['Proporcion_Paro'].mean()

0.06585139836223702

In [34]:
df.query('Shape__Area < @mediana_area')['Proporcion_Paro'].mean()

0.05450009114498015

Entonces, mientras que en autonomías más grandes la proporción de paro es 6.5%, en las más chicas es de 5.4%.

Ejercicio: calculen cómo es esta relación con respecto a la densidad.

In [35]:
## Entonces, mientras que en autonomías más grandes la proporción de paro es 6.5%, en las más chicas es de 5.4%.

###Ejercicio: calculen cómo es esta relación con respecto a la densidad.
df.query('Shape__Area > @mediana_area')['Densidad'].mean()

4.974848681145465e-05

In [36]:
###Ejercicio: calculen cómo es esta relación con respecto a la densidad.
df.query('Densidad <= @mediana_densidad')['Proporcion_Paro'].mean()

0.04953096691700587

## GroupBy: Trabajando sobre grupos

Muchas veces necesitamos analizar métricas, pero sobre agrupamientos de los datos. Por ejemplo, las autonomías se agrupan en provincias y podemos querer ver el desempeño de cada provinicia. O en lugar de analizar la densidad separando únicamente por la mediana, podemos querer ver qué pasa en cada percentil.

En Pandas las operaciones sobre grupos se pueden ver como una combinación de las operaciones Split Apply Combine.

<img src='https://storage.googleapis.com/humai-datasets/aws_s3/analisis_de_datos/3_Agrupacion_y_Agregacion/splitapplycombine.png'></img>

En algunos casos la operación que aplicamos sobre el dataframe original reduce el tamaño del mismo, por ejemplo cuando devolvemos la media de cada grupo y otras veces no, por ejemplo cuando comparamos cada elemento del grupo contra un benchmark del mismo, por ejemplo si quisiéramos hacer un ranking por juego para grupos de jugadores.

## Clases que se encargan de la Agregación en Pandas

## DataSetGroupBy

Veamos qué devuelve pandas cuando agrupamos un dataset por una columna:

In [65]:
df.groupby('PAD_1_COD_PROV')

In [66]:
print(df.groupby('PAD_1_COD_PROV'))

In [67]:
df1 = df.groupby('PAD_1_COD_PROV')

In [68]:
df1

In [64]:
df.groupby('PAD_1_COD_PROV')['Shape__Area']  ## series de groupby

En lugar de recibir una lista o numpy array de grupos, recibimos un objeto del tipo DataFrameGroupBy. Ahora veamos cuánto tarda en ejecutarse este método.

### Lazy Evaluation

In [39]:
df

,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop,Densidad,Proporcion_Paro
0,1,1001,Alegría-Dulantzi,1,16,184.0,2856.0,Araba/Álava,16.0,3.709269e+07,48191.568537,1.0,7.699629e-05,0.064426
1,2,1002,Amurrio,1,16,766.0,10260.0,Araba/Álava,16.0,1.799114e+08,90623.621986,1.0,5.702806e-05,0.074659
2,3,1003,Aramaio,1,16,18.0,1502.0,Araba/Álava,16.0,1.377066e+08,58592.914048,1.0,1.090725e-05,0.011984
3,4,1004,Artziniega,1,16,110.0,1840.0,Araba/Álava,16.0,5.130078e+07,31498.524044,1.0,3.586690e-05,0.059783
4,5,1006,Armiñón,1,16,12.0,234.0,Araba/Álava,16.0,2.380288e+07,33293.805036,1.0,9.830741e-06,0.051282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8165,8166,50901,Biel,50,2,3.0,143.0,Zaragoza,2.0,2.396392e+08,71731.607076,50.0,5.967305e-07,0.020979
8166,8167,50902,Marracos,50,2,2.0,91.0,Zaragoza,2.0,3.102103e+07,27441.268570,50.0,2.933494e-06,0.021978
8167,8168,50903,Villamayor de Gállego,50,2,94.0,2753.0,Zaragoza,2.0,1.627609e+08,63010.590427,50.0,1.691438e-05,0.034145
8168,8169,51001,Ceuta,51,18,12477.0,84519.0,Ceuta,18.0,2.944199e+07,50690.389444,51.0,2.870696e-03,0.147624


In [40]:
%%timeit
df.groupby('PAD_1_COD_PROV')

78.5 µs ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [41]:
%%timeit
a = list(df.groupby('PAD_1_COD_PROV'))

6.34 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Hacer únicamente la agrupación por código de provincia lleva 50 microsegundos pero si convertimos el resultado a una lista, forzamos a que efectivamente se separen los grupos y eso tarda 10.9 ms.  

Este comportamiento se llama "lazy evaluation" y es muy importante en todos los motores de procesamiento de datos. Ejecutar las operaciones computacionalmente pesadas, sólo cuando se necesita permite hacer los procesos más eficientes.

In [71]:
a = list(df.groupby('PAD_1_COD_PROV'))
type(a[0])

tuple

In [73]:
a[0][1]

,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop,Densidad,Proporcion_Paro,Decil_Paro
51,52,2001,Abengibre,2,8,54.0,761.0,Albacete,8.0,5.132677e+07,32589.190350,2.0,0.000015,0.070959,7
52,53,2002,Alatoz,2,8,68.0,582.0,Albacete,8.0,1.061114e+08,48006.643705,2.0,0.000005,0.116838,10
53,54,2003,Albacete,2,8,16352.0,172426.0,Albacete,8.0,1.890078e+09,316172.515745,2.0,0.000091,0.094835,9
54,55,2004,Albatana,2,8,74.0,714.0,Albacete,8.0,5.001554e+07,28900.163978,2.0,0.000014,0.103641,9
55,56,2005,Alborea,2,8,93.0,710.0,Albacete,8.0,1.204546e+08,46911.754532,2.0,0.000006,0.130986,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,134,2083,Villavaliente,2,8,18.0,234.0,Albacete,8.0,5.796096e+07,39334.638806,2.0,0.000004,0.076923,8
134,135,2084,Villaverde de Guadalimar,2,8,54.0,359.0,Albacete,8.0,1.263427e+08,61166.513461,2.0,0.000003,0.150418,10
135,136,2085,Viveros,2,8,30.0,337.0,Albacete,8.0,1.077795e+08,58399.444351,2.0,0.000003,0.089021,9
136,137,2086,Yeste,2,8,369.0,2818.0,Albacete,8.0,8.322489e+08,153920.326022,2.0,0.000003,0.130944,10


In [76]:
a[1]

('Alicante/Alacant',
      OBJECTID  Codigo                 Texto  Cod_Prov  Cod_CCAA  \
 138       139    3001               Adsubia         3        10   
 139       140    3002                 Agost         3        10   
 140       141    3003                 Agres         3        10   
 141       142    3004                Aigües         3        10   
 142       143    3005              Albatera         3        10   
 ..        ...     ...                   ...       ...       ...   
 274       275    3140               Villena         3        10   
 275       276    3901          Poblets, els         3        10   
 276       277    3902  Pilar de la Horadada         3        10   
 277       278    3903       Montesinos, Los         3        10   
 278       279    3904            San Isidro         3        10   
 
      TotalParoRegistrado  PAD_1C02    PAD_1_COD_PROV  PAD_1_COD_CCAA  \
 138                 19.0     689.0  Alicante/Alacant            10.0   
 139           

In [44]:
type(a[0][0])

str

In [45]:
type(a[0][1])

pandas.core.frame.DataFrame

### Iterar sobre los grupos

Podemos recorrer los grupos en un loop for, desempaquetando la tupla que contine el nombre del grupo y el DataFrame correspondiente.

Veamos a ver cuántas autonomías tiene cada provincia.

In [74]:
for (cod_prov, group) in df.groupby('PAD_1_COD_PROV'):
    print("provincia={0}, tamaño={1}".format(cod_prov, group.shape[0]))

provincia=Albacete, tamaño=87
provincia=Alicante/Alacant, tamaño=141
provincia=Almería, tamaño=103
provincia=Araba/Álava, tamaño=51
provincia=Asturias, tamaño=78
provincia=Badajoz, tamaño=165
provincia=Balears, Illes, tamaño=67
provincia=Barcelona, tamaño=311
provincia=Bizkaia, tamaño=112
provincia=Burgos, tamaño=371
provincia=Cantabria, tamaño=102
provincia=Castellón/Castelló, tamaño=135
provincia=Ceuta, tamaño=1
provincia=Ciudad Real, tamaño=102
provincia=Coruña, A, tamaño=93
provincia=Cuenca, tamaño=238
provincia=Cáceres, tamaño=222
provincia=Cádiz, tamaño=44
provincia=Córdoba, tamaño=75
provincia=Gipuzkoa, tamaño=88
provincia=Girona, tamaño=221
provincia=Granada, tamaño=172
provincia=Guadalajara, tamaño=288
provincia=Huelva, tamaño=79
provincia=Huesca, tamaño=202
provincia=Jaén, tamaño=97
provincia=León, tamaño=211
provincia=Lleida, tamaño=231
provincia=Lugo, tamaño=67
provincia=Madrid, tamaño=179
provincia=Melilla, tamaño=1
provincia=Murcia, tamaño=45
provincia=Málaga, tamaño=103


Ejercicio: Calculen la media para cada provincia de la proporción de paro. Midan el tiempo que tarda en ejecutarse esa consulta.

In [83]:
# Convert the relevant columns to numeric type before applying mean()
# Specify the columns you want the mean of, excluding columns with object data types
numeric_columns = ['TotalParoRegistrado', 'PAD_1C02', 'Shape__Area', 'Densidad', 'Proporcion_Paro']  # Replace with your actual numeric columns

df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df.groupby('PAD_1_COD_PROV')[numeric_columns].mean()

,TotalParoRegistrado,PAD_1C02,Shape__Area,Densidad,Proporcion_Paro
PAD_1_COD_PROV,,,,,
Albacete,461.551724,4507.103448,2.830905e+08,0.000009,0.107139
Alicante/Alacant,1209.056738,13024.531915,6.742726e+07,0.000185,0.073767
Almería,610.223301,6837.834951,1.345174e+08,0.000056,0.071866
Araba/Álava,444.882353,6355.411765,1.089912e+08,0.000031,0.046019
Asturias,1105.871795,13366.769231,2.567214e+08,0.000077,0.067554
Badajoz,484.545455,4146.139394,2.169983e+08,0.000024,0.110964
"Balears, Illes",972.716418,16525.671642,1.253382e+08,0.000142,0.047009
Barcelona,1059.122186,17822.122186,4.467244e+07,0.000609,0.050981
Bizkaia,702.035714,10246.214286,3.714319e+07,0.000457,0.051276


In [79]:

df.groupby('PAD_1_COD_PROV')['Proporcion_Paro'].mean()

,Proporcion_Paro
PAD_1_COD_PROV,
Albacete,0.107139
Alicante/Alacant,0.073767
Almería,0.071866
Araba/Álava,0.046019
Asturias,0.067554
Badajoz,0.110964
"Balears, Illes",0.047009
Barcelona,0.050981
Bizkaia,0.051276


In [63]:
#### Ejercicio: Calculen la media para cada provincia de la proporción de paro. Midan el tiempo que tarda en ejecutarse esa consulta.
for (cod_prov, group) in df.groupby('PAD_1_COD_PROV'):
    print("provincia={0}, tamaño={1}".format(cod_prov, group['Proporcion_Paro'].mean()))

provincia=Albacete, tamaño=0.10713858566560387
provincia=Alicante/Alacant, tamaño=0.07376662378228857
provincia=Almería, tamaño=0.07186639744064328
provincia=Araba/Álava, tamaño=0.04601913608875631
provincia=Asturias, tamaño=0.06755422473449564
provincia=Badajoz, tamaño=0.11096359049335525
provincia=Balears, Illes, tamaño=0.04700907407618089
provincia=Barcelona, tamaño=0.05098081879962116
provincia=Bizkaia, tamaño=0.051276205419190306
provincia=Burgos, tamaño=0.04237983801091064
provincia=Cantabria, tamaño=0.07480799327869814
provincia=Castellón/Castelló, tamaño=0.054756219005669675
provincia=Ceuta, tamaño=0.14762361125900686
provincia=Ciudad Real, tamaño=0.11188105881588194
provincia=Coruña, A, tamaño=0.06494319003298168
provincia=Cuenca, tamaño=0.06108539521547999
provincia=Cáceres, tamaño=0.1004069095090072
provincia=Cádiz, tamaño=0.1313158338754046
provincia=Córdoba, tamaño=0.08694518562478057
provincia=Gipuzkoa, tamaño=0.04320205523608764
provincia=Girona, tamaño=0.041608394342017

## SeriesGroupBy

Veamos el tipo de objeto que se forma cuando tomamos una serie del DataFrame.

In [84]:
type(df.groupby('PAD_1_COD_PROV')['Proporcion_Paro'])  ## es una serie groupby es una subclase series

pandas.core.groupby.generic.SeriesGroupBy

Es un objeto de tipo SeriesGroupBy. A estos objetos, podemos aplicarles cualquier función de agregación.

In [48]:
%%time
df.groupby('PAD_1_COD_PROV')['Proporcion_Paro'].mean()

CPU times: user 2.97 ms, sys: 7 µs, total: 2.97 ms
Wall time: 6.68 ms


,Proporcion_Paro
PAD_1_COD_PROV,
Albacete,0.107139
Alicante/Alacant,0.073767
Almería,0.071866
Araba/Álava,0.046019
Asturias,0.067554
Badajoz,0.110964
"Balears, Illes",0.047009
Barcelona,0.050981
Bizkaia,0.051276


Noten que en este caso, la operación sobre el objeto SeriesGroupBy tarda (en este caso, va a depender del hardware) casi 15 veces que el loop for sobre el GroupBy. Esto es gracias a la vectorización que vimos en la clase 1.

In [49]:
a = df.groupby('PAD_1_COD_PROV')['Proporcion_Paro'].mean()

In [50]:
# El resultado de una función de agregación sobre una SeriesGroupBy, es otra serie
type(a)

pandas.core.series.Series

In [86]:
a[0][1]

,OBJECTID,Codigo,Texto,Cod_Prov,Cod_CCAA,TotalParoRegistrado,PAD_1C02,PAD_1_COD_PROV,PAD_1_COD_CCAA,Shape__Area,Shape__Length,Codigop,Densidad,Proporcion_Paro,Decil_Paro
51,52,2001,Abengibre,2,8,54.0,761.0,Albacete,8.0,5.132677e+07,32589.190350,2.0,0.000015,0.070959,7
52,53,2002,Alatoz,2,8,68.0,582.0,Albacete,8.0,1.061114e+08,48006.643705,2.0,0.000005,0.116838,10
53,54,2003,Albacete,2,8,16352.0,172426.0,Albacete,8.0,1.890078e+09,316172.515745,2.0,0.000091,0.094835,9
54,55,2004,Albatana,2,8,74.0,714.0,Albacete,8.0,5.001554e+07,28900.163978,2.0,0.000014,0.103641,9
55,56,2005,Alborea,2,8,93.0,710.0,Albacete,8.0,1.204546e+08,46911.754532,2.0,0.000006,0.130986,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,134,2083,Villavaliente,2,8,18.0,234.0,Albacete,8.0,5.796096e+07,39334.638806,2.0,0.000004,0.076923,8
134,135,2084,Villaverde de Guadalimar,2,8,54.0,359.0,Albacete,8.0,1.263427e+08,61166.513461,2.0,0.000003,0.150418,10
135,136,2085,Viveros,2,8,30.0,337.0,Albacete,8.0,1.077795e+08,58399.444351,2.0,0.000003,0.089021,9
136,137,2086,Yeste,2,8,369.0,2818.0,Albacete,8.0,8.322489e+08,153920.326022,2.0,0.000003,0.130944,10


In [51]:
# El índice de la serie son todas las provincias.
a.index

Index(['Albacete', 'Alicante/Alacant', 'Almería', 'Araba/Álava', 'Asturias',
       'Badajoz', 'Balears, Illes', 'Barcelona', 'Bizkaia', 'Burgos',
       'Cantabria', 'Castellón/Castelló', 'Ceuta', 'Ciudad Real', 'Coruña, A',
       'Cuenca', 'Cáceres', 'Cádiz', 'Córdoba', 'Gipuzkoa', 'Girona',
       'Granada', 'Guadalajara', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'Lugo', 'Madrid', 'Melilla', 'Murcia', 'Málaga', 'Navarra', 'Ourense',
       'Palencia', 'Palmas, Las', 'Pontevedra', 'Rioja, La', 'Salamanca',
       'Santa Cruz de Tenerife', 'Segovia', 'Sevilla', 'Soria', 'Tarragona',
       'Teruel', 'Toledo', 'Valencia/València', 'Valladolid', 'Zamora',
       'Zaragoza', 'Ávila'],
      dtype='object', name='PAD_1_COD_PROV')

Ahora podemos probar otras funciones de agregación, por ejemplo calculemos el área por provincia:

In [52]:
df.groupby('PAD_1_COD_PROV')['Shape__Area'].sum()

,Shape__Area
PAD_1_COD_PROV,
Albacete,2.462887e+10
Alicante/Alacant,9.507244e+09
Almería,1.385529e+10
Araba/Álava,5.558551e+09
Asturias,2.002427e+10
Badajoz,3.580472e+10
"Balears, Illes",8.397659e+09
Barcelona,1.389313e+10
Bizkaia,4.160037e+09


## Agregaciones múltiples

Veamos ahora quiero combinar para cada provincia, cuál es el área, cuál es la población, cuál es la media y la mediana de paro y cuántas autonomías la componen. Para esto existe la función aggregate. Aplicada sobre un DataFrameGroupBy recibe como parámetro un diccionario con las nombres de columnas como claves y el tipo de agregación a realizar como valores. Pueden ser valores únicos o listas con varios valores.


In [53]:
df.groupby('PAD_1_COD_PROV').aggregate({'Shape__Area':'sum',
                                        'PAD_1C02':'sum',
                                        'Proporcion_Paro':['mean','median','size']}).head()

Shape__Area   PAD_1C02 Proporcion_Paro               
                           sum        sum            mean    median size
PAD_1_COD_PROV                                                          
Albacete          2.462887e+10   392118.0        0.107139  0.104993   87
Alicante/Alacant  9.507244e+09  1836459.0        0.073767  0.072584  141
Almería           1.385529e+10   704297.0        0.071866  0.069767  103
Araba/Álava       5.558551e+09   324126.0        0.046019  0.043350   51
Asturias          2.002427e+10  1042608.0        0.067554  0.067519   78

## MultiIndex

Cuando agregamos por varias columnas distintas, nos queda un MultiIndex en las columnas


In [54]:
df_agregado = df.groupby('PAD_1_COD_PROV').aggregate({'Shape__Area':'sum',
                                        'PAD_1C02':'sum',
                                        'Proporcion_Paro':['mean','median','size']})

In [55]:
df_agregado.columns

MultiIndex([(    'Shape__Area',    'sum'),
            (       'PAD_1C02',    'sum'),
            ('Proporcion_Paro',   'mean'),
            ('Proporcion_Paro', 'median'),
            ('Proporcion_Paro',   'size')],
           )

#### Entendiendo la función pd.qcut

También podemos agrupar por más de una columna, usando una lista en lugar de un único valor como parámetro del groupby.

Calculemos los deciles de paro regitrado en los distintos municipios.
Para eso vamos a construir una columna que contenga el decil que ocupa el municipio en el ranking de proporción de paro. El decil 1 va a representar las autonomías que mejor se desempeñan y el decil 10 las que peor lo hacen.

In [87]:
df['Decil_Paro'] = pd.qcut(df['Proporcion_Paro'], 10, labels=range(1,11))  ## QCUT CORTAR POR CANTIDADES , Y LABEL DE 1 A 10


In [88]:
df['Decil_Paro']

,Decil_Paro
0,6
1,8
2,1
3,6
4,5
...,...
8165,2
8166,2
8167,3
8168,10


Ahora podemos agrupar por provincia y decil para ver cuántos municipios de cada decil hay en cada provincia.

In [57]:
serie_prov_decil = df.groupby(['PAD_1_COD_PROV','Decil_Paro']).size()

<ipython-input-57-4929e645c00e>:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [90]:
serie_prov_decil  ## UBICAR EN PAQUETS IGUALES A LOS MUNICIPIOS

PAD_1_COD_PROV  Decil_Paro
Albacete        1              0
                2              0
                3              2
                4              0
                5              1
                              ..
Ávila           6             24
                7             28
                8             21
                9             21
                10            24
Length: 520, dtype: int64

In [58]:
type(serie_prov_decil)

pandas.core.series.Series

In [59]:
serie_prov_decil.index

MultiIndex([('Albacete',  1),
            ('Albacete',  2),
            ('Albacete',  3),
            ('Albacete',  4),
            ('Albacete',  5),
            ('Albacete',  6),
            ('Albacete',  7),
            ('Albacete',  8),
            ('Albacete',  9),
            ('Albacete', 10),
            ...
            (   'Ávila',  1),
            (   'Ávila',  2),
            (   'Ávila',  3),
            (   'Ávila',  4),
            (   'Ávila',  5),
            (   'Ávila',  6),
            (   'Ávila',  7),
            (   'Ávila',  8),
            (   'Ávila',  9),
            (   'Ávila', 10)],
           names=['PAD_1_COD_PROV', 'Decil_Paro'], length=520)

In [60]:
serie_prov_decil.head(10)

PAD_1_COD_PROV  Decil_Paro
Albacete        1              0
                2              0
                3              2
                4              0
                5              1
                6              3
                7              4
                8              6
                9             29
                10            42
dtype: int64

El objeto MultiIndex puede ser complicado para trabajar. Por eso conviene utilizar el método reset_index() para volver a un DataFrame común.

In [61]:
df_prov = serie_prov_decil.reset_index()

In [91]:
df_prov

,PAD_1_COD_PROV,Decil_Paro,0
0,Albacete,1,0
1,Albacete,2,0
2,Albacete,3,2
3,Albacete,4,0
4,Albacete,5,1
...,...,...,...
515,Ávila,6,24
516,Ávila,7,28
517,Ávila,8,21
518,Ávila,9,21


In [62]:
df_prov.head()

,PAD_1_COD_PROV,Decil_Paro,0
0,Albacete,1,0
1,Albacete,2,0
2,Albacete,3,2
3,Albacete,4,0
4,Albacete,5,1


## Ejercicios

Ahora vamos a querer trabajar a nivel de provincia. Calculen la densidad, la proporción de paro y la cantidad de municipios para cada provincia ¿Cuál es la provincia con mayor proporción de paro? ¿Cuál es la que tiene menos?

In [99]:
## Calculen la densidad, la proporción de paro y la cantidad de municipios para cada provincia ¿Cuál es la provincia con mayor proporción de paro? ¿Cuál es la que tiene menos?
df_prov1 = df.groupby('PAD_1_COD_PROV').aggregate({'Shape__Area':'sum',
                                        'Densidad': 'mean',
                                        'PAD_1C02':'sum',
                                        'Codigo':'count',
                                        'Proporcion_Paro':['mean','median','size']})

In [100]:
df_prov1

Shape__Area  Densidad   PAD_1C02 Codigo  \
                                 sum      mean        sum  count   
PAD_1_COD_PROV                                                     
Albacete                2.462887e+10  0.000009   392118.0     87   
Alicante/Alacant        9.507244e+09  0.000185  1836459.0    141   
Almería                 1.385529e+10  0.000056   704297.0    103   
Araba/Álava             5.558551e+09  0.000031   324126.0     51   
Asturias                2.002427e+10  0.000077  1042608.0     78   
Badajoz                 3.580472e+10  0.000024   684113.0    165   
Balears, Illes          8.397659e+09  0.000142  1107220.0     67   
Barcelona               1.389313e+10  0.000609  5542680.0    311   
Bizkaia                 4.160037e+09  0.000457  1147576.0    112   
Burgos                  2.620377e+10  0.000009   360995.0    371   
Cantabria               1.002555e+10  0.000121   582206.0    102   
Castellón/Castelló      1.139656e+10  0.000054   579245.0    135   
Ceuta                   2.944199e+07  0.002871    84519.0      1   
Ciudad Real             3.278477e+10  0.000016   506888.0    102   
Coruña, A               1.493245e+10  0.000114  1122799.0     93   
Cuenca                  2.915608e+10  0.000005   201071.0    238   
Cáceres                 3.358043e+10  0.000012   402835.0    222   
Cádiz                   1.154687e+10  0.000337  1239889.0     44   
Córdoba                 2.221049e+10  0.000042   791610.0     75   
Gipuzkoa                3.757743e+09  0.000190   717832.0     88   
Girona                  1.075165e+10  0.000101   753576.0    221   
Granada                 2.002997e+10  0.000151   915392.0    172   
Guadalajara             2.134149e+10  0.000014   252882.0    288   
Huelva                  1.615329e+10  0.000046   519596.0     79   
Huesca                  2.852289e+10  0.000008   221079.0    202   
Jaén                    2.178012e+10  0.000030   648250.0     97   
León                    2.879265e+10  0.000024   473604.0    211   
Lleida                  2.209015e+10  0.000031   434041.0    231   
Lugo                    1.844736e+10  0.000027   336527.0     67   
Madrid                  1.390127e+10  0.000291  6466996.0    179   
Melilla                 1.993434e+07  0.004315    86026.0      1   
Murcia                  1.825000e+10  0.000196  1464847.0     45   
Málaga                  1.142476e+10  0.000169  1629298.0    103   
Navarra                 1.779043e+10  0.000132   640647.0    272   
Ourense                 1.326417e+10  0.000030   314853.0     92   
Palencia                1.476077e+10  0.000009   164644.0    191   
Palmas, Las             5.250341e+09  0.000359  1097800.0     34   
Pontevedra              8.246811e+09  0.000149   944346.0     62   
Rioja, La               9.220822e+09  0.000023   315794.0    174   
Salamanca               2.157665e+10  0.000019   335985.0    362   
Santa Cruz de Tenerife  4.370210e+09  0.000265  1004124.0     54   
Segovia                 1.222843e+10  0.000010   155652.0    209   
Sevilla                 2.230211e+10  0.000323  1939775.0    105   
Soria                   1.845798e+10  0.000003    90040.0    183   
Tarragona               1.110738e+10  0.000091   792299.0    184   
Teruel                  2.576417e+10  0.000003   136977.0    236   
Toledo                  2.606408e+10  0.000038   688672.0    204   
Valencia/València       1.811013e+10  0.000510  2544264.0    266   
Valladolid              1.453133e+10  0.000020   523679.0    225   
Zamora                  1.897687e+10  0.000008   180406.0    248   
Zaragoza                3.093779e+10  0.000023   950507.0    293   
Ávila                   1.396637e+10  0.000008   162514.0    248   

                       Proporcion_Paro                 
                                  mean    median size  
PAD_1_COD_PROV                                         
Albacete                      0.107139  0.104993   87  
Alicante/Alacant              0.073767  0.072584  141  
Alme

In [98]:
## Calculen la densidad, la proporción de paro y la cantidad de municipios para cada provincia
df_prov1['Densidad'] = df_prov1['PAD_1C02'] / df_prov1['Shape__Area']

In [96]:
df_prov1['Densidad']

,Densidad
PAD_1_COD_PROV,
Albacete,0.000016
Alicante/Alacant,0.000193
Almería,0.000051
Araba/Álava,0.000058
Asturias,0.000052
Badajoz,0.000019
"Balears, Illes",0.000132
Barcelona,0.000399
Bizkaia,0.000276


In [94]:
df_prov1

Shape__Area   PAD_1C02 Proporcion_Paro               
                                 sum        sum            mean    median size
PAD_1_COD_PROV                                                                
Albacete                2.462887e+10   392118.0        0.107139  0.104993   87
Alicante/Alacant        9.507244e+09  1836459.0        0.073767  0.072584  141
Almería                 1.385529e+10   704297.0        0.071866  0.069767  103
Araba/Álava             5.558551e+09   324126.0        0.046019  0.043350   51
Asturias                2.002427e+10  1042608.0        0.067554  0.067519   78
Badajoz                 3.580472e+10   684113.0        0.110964  0.111384  165
Balears, Illes          8.397659e+09  1107220.0        0.047009  0.046549   67
Barcelona               1.389313e+10  5542680.0        0.050981  0.050098  311
Bizkaia                 4.160037e+09  1147576.0        0.051276  0.049558  112
Burgos                  2.620377e+10   360995.0        0.042380  0.039216  371
Cantabria               1.002555e+10   582206.0        0.074808  0.075883  102
Castellón/Castelló      1.139656e+10   579245.0        0.054756  0.053381  135
Ceuta                   2.944199e+07    84519.0        0.147624  0.147624    1
Ciudad Real             3.278477e+10   506888.0        0.111881  0.112277  102
Coruña, A               1.493245e+10  1122799.0        0.064943  0.065204   93
Cuenca                  2.915608e+10   201071.0        0.061085  0.061931  238
Cáceres                 3.358043e+10   402835.0        0.100407  0.100555  222
Cádiz                   1.154687e+10  1239889.0        0.131316  0.129083   44
Córdoba                 2.221049e+10   791610.0        0.086945  0.086957   75
Gipuzkoa                3.757743e+09   717832.0        0.043202  0.044333   88
Girona                  1.075165e+10   753576.0        0.041608  0.038948  221
Granada                 2.002997e+10   915392.0        0.083447  0.082217  172
Guadalajara             2.134149e+10   252882.0        0.047564  0.048535  288
Huelva                  1.615329e+10   519596.0        0.105262  0.106219   79
Huesca                  2.852289e+10   221079.0        0.031334  0.030907  202
Jaén                    2.178012e+10   648250.0        0.056368  0.051306   97
León                    2.879265e+10   473604.0        0.057527  0.057471  211
Lleida                  2.209015e+10   434041.0        0.034761  0.032541  231
Lugo                    1.844736e+10   336527.0        0.051197  0.048796   67
Madrid                  1.390127e+10  6466996.0        0.067928  0.068581  179
Melilla                 1.993434e+07    86026.0        0.126334  0.126334    1
Murcia                  1.825000e+10  1464847.0        0.083757  0.082504   45
Málaga                  1.142476e+10  1629298.0        0.089687  0.087475  103
Navarra                 1.779043e+10   640647.0        0.045826  0.045609  272
Ourense                 1.326417e+10   314853.0        0.063683  0.060270   92
Palencia                1.476077e+10   164644.0        0.041849  0.040541  191
Palmas, Las             5.250341e+09  1097800.0        0.098515  0.103516   34
Pontevedra              8.246811e+09   944346.0        0.087798  0.090540   62
Rioja, La               9.220822e+09   315794.0        0.035615  0.035044  174
Salamanca               2.157665e+10   335985.0        0.060401  0.059701  362
Santa Cruz de Tenerife  4.370210e+09  1004124.0        0.106326  0.109498   54
Segovia                 1.222843e+10   155652.0        0.040490  0.037383  209
Sevilla                 2.230211e+10  1939775.0        0.098427  0.099086  105
Soria                   1.845798e+10    90040.0        0.026752  0.025641  183
Tarragona               1.110738e+10   792299.0        0.053319  0.051238  184
Teruel                  2.576417e+10   136977.0        0.038627  0.035577  236
Toledo                  2.606408e+10   688672.0        0.103076  0.102512  204
Valencia/València       1.811013e+10  2544264.0        0.069956  0.069269  

In [101]:
df_resultado = df.groupby('PAD_1_COD_PROV').aggregate({'Shape__Area':'sum',
                                        'Densidad': 'mean',
                                        'Codigo': 'count',
                                        'PAD_1C02':'sum',
                                        'Proporcion_Paro':['mean','median','size']})

In [102]:
df_resultado

Shape__Area  Densidad Codigo   PAD_1C02  \
                                 sum      mean  count        sum   
PAD_1_COD_PROV                                                     
Albacete                2.462887e+10  0.000009     87   392118.0   
Alicante/Alacant        9.507244e+09  0.000185    141  1836459.0   
Almería                 1.385529e+10  0.000056    103   704297.0   
Araba/Álava             5.558551e+09  0.000031     51   324126.0   
Asturias                2.002427e+10  0.000077     78  1042608.0   
Badajoz                 3.580472e+10  0.000024    165   684113.0   
Balears, Illes          8.397659e+09  0.000142     67  1107220.0   
Barcelona               1.389313e+10  0.000609    311  5542680.0   
Bizkaia                 4.160037e+09  0.000457    112  1147576.0   
Burgos                  2.620377e+10  0.000009    371   360995.0   
Cantabria               1.002555e+10  0.000121    102   582206.0   
Castellón/Castelló      1.139656e+10  0.000054    135   579245.0   
Ceuta                   2.944199e+07  0.002871      1    84519.0   
Ciudad Real             3.278477e+10  0.000016    102   506888.0   
Coruña, A               1.493245e+10  0.000114     93  1122799.0   
Cuenca                  2.915608e+10  0.000005    238   201071.0   
Cáceres                 3.358043e+10  0.000012    222   402835.0   
Cádiz                   1.154687e+10  0.000337     44  1239889.0   
Córdoba                 2.221049e+10  0.000042     75   791610.0   
Gipuzkoa                3.757743e+09  0.000190     88   717832.0   
Girona                  1.075165e+10  0.000101    221   753576.0   
Granada                 2.002997e+10  0.000151    172   915392.0   
Guadalajara             2.134149e+10  0.000014    288   252882.0   
Huelva                  1.615329e+10  0.000046     79   519596.0   
Huesca                  2.852289e+10  0.000008    202   221079.0   
Jaén                    2.178012e+10  0.000030     97   648250.0   
León                    2.879265e+10  0.000024    211   473604.0   
Lleida                  2.209015e+10  0.000031    231   434041.0   
Lugo                    1.844736e+10  0.000027     67   336527.0   
Madrid                  1.390127e+10  0.000291    179  6466996.0   
Melilla                 1.993434e+07  0.004315      1    86026.0   
Murcia                  1.825000e+10  0.000196     45  1464847.0   
Málaga                  1.142476e+10  0.000169    103  1629298.0   
Navarra                 1.779043e+10  0.000132    272   640647.0   
Ourense                 1.326417e+10  0.000030     92   314853.0   
Palencia                1.476077e+10  0.000009    191   164644.0   
Palmas, Las             5.250341e+09  0.000359     34  1097800.0   
Pontevedra              8.246811e+09  0.000149     62   944346.0   
Rioja, La               9.220822e+09  0.000023    174   315794.0   
Salamanca               2.157665e+10  0.000019    362   335985.0   
Santa Cruz de Tenerife  4.370210e+09  0.000265     54  1004124.0   
Segovia                 1.222843e+10  0.000010    209   155652.0   
Sevilla                 2.230211e+10  0.000323    105  1939775.0   
Soria                   1.845798e+10  0.000003    183    90040.0   
Tarragona               1.110738e+10  0.000091    184   792299.0   
Teruel                  2.576417e+10  0.000003    236   136977.0   
Toledo                  2.606408e+10  0.000038    204   688672.0   
Valencia/València       1.811013e+10  0.000510    266  2544264.0   
Valladolid              1.453133e+10  0.000020    225   523679.0   
Zamora                  1.897687e+10  0.000008    248   180406.0   
Zaragoza                3.093779e+10  0.000023    293   950507.0   
Ávila                   1.396637e+10  0.000008    248   162514.0   

                       Proporcion_Paro                 
                                  mean    median size  
PAD_1_COD_PROV                                         
Albacete                      0.107139  0.104993   87  
Alicante/Alacant              0.073767  0.072584  141  
Alme

In [103]:
df_resultado.iloc[0,2]

87

In [104]:
df_resultado.iloc[-1,2]

248

In [105]:
df_resultado.index[0]

'Albacete'

In [106]:
df_resultado.index[-1]

'Ávila'